<a href="https://colab.research.google.com/github/toussaintma/petitproust/blob/main/pp_prompt_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub

In [ ]:
# model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=500,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
prompt_template = """
<|system|>

You are an expert writer writing in french. Rewrite this text in the litterary style of author Marcel Proust. Write exclusively in french. Use the following example texts of his writing to help:

{examples}

</s>
<|user|>

{source}

</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["examples", "source"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [ ]:
original_text = """
Chaque soir je m'endors en rêvant de fusée. Dans mes rêves je suis un cosmonaute dans une fusée partant pour la lune et je rêve.
"""
answer_llmonly = llm_chain.invoke({"examples": "", "source": original_text})

In [ ]:
print(answer_llmonly)